In [4]:
%%capture --no-stderr

# 사전 세팅
%pip install langchain langchainhub langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community
%pip install --upgrade --quiet  nest_asyncio
%pip install faiss-cpu

import os

key_path = '../openai_key.txt'
with open(key_path, 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

In [6]:
# 0. RAG 벡터 생성
import nest_asyncio
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

nest_asyncio.apply()

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

loader = WebBaseLoader(urls)
loader.requests_per_second = 1
documents = loader.aload()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(texts, embeddings)

retriever = vectorstore.as_retriever()

Fetching pages: 100%|##########| 3/3 [00:00<00:00,  6.55it/s]


In [8]:
# 1. 쿼리 입력
query = "What is the capital of Korea?"

In [16]:
# 2. Relevance Checker
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def get_relevance_score(query: str):
    prompt = hub.pull("rlm/rag-document-relevance")
    chain = (
            {"input": {"question": RunnablePassthrough(), "documents": retriever | format_docs}}
            | prompt
            | llm
            | StrOutputParser()
    )
    return chain.invoke(query)


True


In [68]:
if (get_relevance_score(query) == "0"):
    print("No")


def generate_answer(query: str):
    prompt = hub.pull("rlm/rag-prompt")
    chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
    )
    return chain.invoke(query)



No
content='The capital of Korea is Seoul.' response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 500, 'total_tokens': 507}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'stop', 'logprobs': None} id='run-fb00077f-8196-437d-a4b0-dc2d88c7dac5-0' usage_metadata={'input_tokens': 500, 'output_tokens': 7, 'total_tokens': 507}


In [74]:
# check hallucination
def check_hallucination(query: str, output: str):
    prompt = hub.pull("rlm/rag-answer-hallucination")
    chain = (
            {"input": {"documents": retriever | format_docs}, "output": {}}
            | prompt
            | llm
            | StrOutputParser()
    )
    return chain.invoke(query)


check_hallucination(query, "output")

'0'

In [73]:
def run(query: str):
    relevance_score = get_relevance_score(query)
    if (relevance_score == "0"):
        return "No"

    answer = generate_answer(query)

    if (check_hallucination(query, answer) == "1"):
        return generate_answer(query)
    return answer


print(run("RAG 에 대한 저자의 생각은 무엇인가?"))
print(run("오늘 점심 메뉴는 무엇인가?"))

content='저자는 RAG 방식이 다양한 확률 기반 접근 방식 중에서 상대적으로 덜 효과적이라는 결론을 내립니다. 실험에 따르면, PoE가 RAG보다 더 높은 성능을 보이며, RAG의 개별 확률이 다른 방식에 비해 덜 정보적임을 시사합니다. 또한, 모델이 최신 정보에 접근할 수 있음에도 불구하고, 최신 질문에 대한 성능이 떨어진다는 점도 지적하고 있습니다.' response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 831, 'total_tokens': 933}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'stop', 'logprobs': None} id='run-d5f4eb8e-b67c-48f1-8bda-1d126a22619d-0' usage_metadata={'input_tokens': 831, 'output_tokens': 102, 'total_tokens': 933}
No


input_variables=['input', 'output'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-answer-hallucination', 'lc_hub_commit_hash': 'a88d01cb864e906293aae38575a85627b3b932d6ba2210de4bd69cea9bfd99ab'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n\nGive a binary score 1 or 0, where 1 means that the answer is grounded in / supported by the set of facts.', template_format='mustache')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'output'], template='Facts: {{input.documents}} \n\nLLM generation: {{output}}', template_format='mustache'))] schema_={'type': 'object', 'title': 'Criteria', 'required': ['Score', 'Explanation'], 'properties': {'Score': {'type': 'integer', 'description': 'Is the LLM generation grounded in the Facts?'}, 'Explanation': {'type': 'string', 'description': 'Explain your reasoning for 